In [15]:
#-- Import Libraries ---------------------------------------------------------------------------------
import torch
import torch.nn as nn
from torchvision import models, transforms
import cv2
import numpy as np
import os
import glob
from PIL import Image
import shutil
#-----------------------------------------------------------------------------------------------------

In [16]:
#-- Load Model -----------------------------------------------------------------------------------
resnet = models.resnet50(pretrained=True)
resnet.eval()  
#-----------------------------------------------------------------------------------------------------

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
#-- Set Transform for Frames --------------------------------------------------------------------------
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
#-----------------------------------------------------------------------------------------------------

In [23]:
#-- Function to Extract features for Frames -------------------------------------------------------------
def extract_features(frame):   
    image = Image.fromarray(frame)
    image = transform(image).unsqueeze(0)  
    with torch.no_grad():
        features = resnet(image)  
    return features.squeeze(0).cpu().numpy() 
#-----------------------------------------------------------------------------------------------------

In [19]:
#-- Function to Extract and Save Features for all Frames -------------------------------------------------
def process_video(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    frame_features = []
    frame_indices = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        
        features = extract_features(frame)
        frame_features.append(features)
        frame_indices.append(cap.get(cv2.CAP_PROP_POS_FRAMES)) 

    cap.release()

    frame_features = np.array(frame_features)
    
    video_name = os.path.basename(video_path).split('.')[0]  
    output_file = os.path.join(output_dir, f"{video_name}_features.npy")
    np.save(output_file, frame_features)  

    print(f"Features for {video_name} saved to {output_file}")
#-----------------------------------------------------------------------------------------------------

In [21]:
#-- Run on Burglary Samples ----------------------------------------------------------------------------
burglary_samples_dir = "/kaggle/input/novin-create-binary-burglary-ds/burglary_samples/"  
burglary_output_dir = "burglary_samples"  

os.makedirs(burglary_output_dir, exist_ok=True)

video_files = glob.glob(os.path.join(burglary_samples_dir, "*.mp4"))  
print(f'Number of Videos: {len(video_files)}')

for video_file in video_files:
    process_video(video_file, burglary_output_dir)
    

#-- zip output--
output_filename = 'burglary_samples'

shutil.make_archive(output_filename, 'zip', burglary_output_dir)
shutil.rmtree(burglary_output_dir)
#-----------------------------------------------------------------------------------------------------

Number of Videos: 34
Features for burglary_24 saved to burglary_samples/burglary_24_features.npy


In [22]:
#-- Run on Not Burglary Samples ----------------------------------------------------------------------------
not_burglary_samples_dir = "/kaggle/input/novin-create-binary-burglary-ds/not_burglary_samples/"  
not_burglary_output_dir = "not_burglary_samples"  

os.makedirs(not_burglary_output_dir, exist_ok=True)

video_files = glob.glob(os.path.join(not_burglary_samples_dir, "*.mp4"))  
print(f'Number of Videos: {len(video_files)}')

for video_file in video_files:
    process_video(video_file, not_burglary_output_dir)
    

#-- zip output--
output_filename = 'not_burglary_samples'

shutil.make_archive(output_filename, 'zip', not_burglary_output_dir)
shutil.rmtree(not_burglary_output_dir)
#-----------------------------------------------------------------------------------------------------

Number of Videos: 34
Features for not_burglary_28 saved to not_burglary_samples/not_burglary_28_features.npy
